# RNAseq analysis using edgeR and limma voom 
#### created by Charity Washam and Stephanie Byrum

### The data has been trimmed, aligned to the reference genome, and counts matrix generated. 
### The first step of Differential Expression Analysis is to load the sample information and the counts matrix.
#### nf-core RNAseq pipeline shows the workflow to generate the counts matrix
**Reference:**  doi: 10.5281/zenodo.1400710 </br>
Ewels PA, Peltzer A, Fillinger S, Alneberg JA, Patel H, Wilm A, Garcia MU, Di Tommaso P, Nahnsen S. nf-core: Community curated bioinformatics pipelines. bioRxiv. 2019. p. 610741. doi: 10.1101/610741. 

<img src ="https://raw.githubusercontent.com/nf-core/rnaseq/3.8.1/docs/images/nf-core-rnaseq_metro_map_grey.png" alt ="nf-core">

In [ ]:
# create conda environment in Terminal 
conda create -n RNASeq
conda activate RNASeq

conda install r-base bioconductor-edger r-matrixStats bioconductor-annotationdbi bioconductor-org.mm.eg.db bioconductor-glimma -c conda-forge -c bioconda -c r -c defaults
conda install jupyterlab -c conda-forge -c bioconda -c defaults
conda install r-irkernel -c conda-forge -c bioconda -c defaults

In [ ]:
# To install R Kernel for the Jupyter Notebook, use the following command from the PSSC Labs terminal. 
# open R in terminal
R
install.packages("devtools")
devtools::install_github("IRkernel/IRkernel")
IRkernel::installspec()

In [ ]:
# if packages have not been installed using conda
# Install necessary packages into R environment
install.packages("matrixStats")
install.packages("BiocManager")
library("BiocManager")
BiocManager::install("edgeR", quietly=TRUE)
BiocManager::install("AnnotationDbi", quietly=TRUE)
BiocManager::install("org.Mm.eg.db", quietly=TRUE)
BiocManager::install("Glimma")

In [ ]:
library(edgeR)

In [ ]:
# import targets and counts
# load example data
targets <- read.csv("targets.csv", header=TRUE, stringsAsFactors=FALSE)
targets

# Load example data
GenewiseCounts <- read.csv("htseq_counts_12.csv", header=TRUE, row.names=1, stringsAsFactors=FALSE)
head(GenewiseCounts)

## We must define the sample group as factors and set the levels option so the group is not changed to alphabetical order. Also, define the batch column or any other factors.

In [ ]:
# define main factors for the analysis (group main factor for analysis, batch, etc)
targets$group <- factor(targets$group, levels=c(unique(targets$group)))
table(targets$group)
levels(targets$group)

### This data does not include a batch but has different tissues
targets$batch <- factor(as.character(targets$batch), levels=c(unique(targets$batch)))
table(targets$batch)
levels(targets$batch)

In [ ]:
# create data.frame with factors (potential factors for the analysis)
#my_targets <- data.frame(group=targets$group, batch=targets$batch)

my_targets <- data.frame(group=targets$group, tissue=targets$tissue)
rownames(my_targets) <- targets$sample
my_targets

table(my_targets$group)
table(my_targets$tissue)

### DGEList() = Creates a DGEList object from a table of counts (rows=features, columns=samples), group indicator for each column, library size (optional) and a table of feature annotation (optional).

In [ ]:
help("DGEList-class")

In [ ]:
# create DGEList()
y <- DGEList(counts=GenewiseCounts, group=  targets$group, genes=rownames(GenewiseCounts), remove.zeros=FALSE)
colnames(y$genes)[1] <- "ensembl_id"
head(y$counts)[,1:5]
head(y$genes)
head(y$samples)


### Load the gene annotation from the Mus musculus database and add to the DGEList object

In [ ]:
library(AnnotationDbi, quietly=TRUE)
library(org.Mm.eg.db, quietly=TRUE)

In [ ]:
# check the database keytpyes for the specific spelling of IDs
org.Mm.eg.db
keytypes(org.Mm.eg.db)

In [ ]:
### change the keytype since the input counts matrix includes ENSEMBL ids and not SYMBOLs 

y$genes$ENSEMBL  <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "SYMBOL", column="ENSEMBL", multiVals="first")
y$genes$ENTREZID <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "SYMBOL", column="ENTREZID", multiVals="first")
y$genes$SYMBOL   <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "SYMBOL", column="SYMBOL", multiVals="first")
y$genes$GENENAME <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "SYMBOL", column="GENENAME", multiVals="first")

#y$genes$ENSEMBL  <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "ENSEMBL", column="ENSEMBL", multiVals="first")
#y$genes$ENTREZID <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "ENSEMBL", column="ENTREZID", multiVals="first")
#y$genes$SYMBOL   <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "ENSEMBL", column="SYMBOL", multiVals="first")
#y$genes$GENENAME <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "ENSEMBL", column="GENENAME", multiVals="first")

# view final imported data
head(y$counts)[,1:5]
head(y$samples)
head(y$genes)


### Filter genes that contain zeros in all samples

In [ ]:
head(y)

In [ ]:
# example subset
# d <- matrix(rnbinom(16,size=1,mu=10),4,4)
# rownames(d) <- c("a","b","c","d")
# colnames(d) <- c("A1","A2","B1","B2")
# d <- DGEList(counts=d,group=factor(c("A","A","B","B")))
#d[1:2,]
#d[1:2,2]
# d[,3]

In [ ]:
colnames(y)

In [ ]:
## [2] REMOVE GENES WITH ZERO COUNTS

# genes with zero counts in all samples
zeros <- rowSums(y$counts)==0
table(zeros)

# keep genes with at least 1 count in 1 sample (i.e. filter genes with zero counts in all samples). 
keep <- rowSums(y$counts) > 0
table(keep)
y <- y[keep,, keep.lib.sizes=FALSE]
dim(y)


<b>Dealing with Sequencing Bias: </b>Check raw values such as number of groups, samples, genes to make sure the object size is correct. Check the library size for each sample, which indicates the depth of sequencing. Calculate counts per million.  

<p>The differences between the number of readings is due to accidental variations in how much each different library is loaded into the flowcell and sequenced. </p>

<p>When loading a multiplexed RNAseq experiment into the flowcell, one quantifies the DNA (the initial RNA is reverse transcribed to DNA) amount for each library, "normalizes" the libraries (i.e., dilutes all libraries to the same DNA concentration), and loads the same amount of each library into the flowcell. In an ideal world, all libraries would have the same number of reads, and then no library size normalization would be necessary for the analysis - this is rarely the case, though, and there is substantial reads number variation between libraries. https://www.biostars.org/p/349881/#350181</p>

<p> There are multiple bias engaged in RNAseq experiment : <b>library size, genes length, RNA population composition for each condition and genes GC composition.</b>
Two bias can be discarded if you compare genes amongst conditions only, because these two are inherent to the gene : <b>genes length and genes GC composition</b></p>
    
<p><b>genes length :</b> The raw count of two genes cannot compared if gene A is twice longer than gene B. Due to its length, the longest gene will have higher chance to be sequenced than the short one. And in the end, for the same expression level, the longest gene will identify more reads than the shortest one.</p>

<p><b>genes GC composition :</b> For two genes with different GC content, the one with the closest GC content to 40% will more likely be sequenced. </p>
<p>The others bias are "technical bias", due to your sample and sequencing method.
<b>library size : </b> The most well know bias. You create two libraries for two conditions with the same RNA composition. The second library works way better than the first one, you get 12 000 000 reads for condition A and 36 000 000 reads for condition B. You will have three times (36 000 000/12 000 000 = 3) more of each RNA in your condition B than your condition A. </p>

<img src="https://i.ibb.co/Ld4YH9m/condition-A.png" alt="condition A">

<img src="https://i.ibb.co/sgF0PSj/condition-B.png" alt="condition B">
  
    
<p>Apart from the differences in library depth, an additional problem is that RNASeq frequently have different amounts of different RNA types in them. A simple example could be that you have more rRNA in one sample than in another (lets say 1% vs 20%) if you do not take this into account it would look like the majority of protein coding genes were downregulated simply because they would get a smaller fraction of reads. Such effects is handled by doing a inter-library normalization an analysis build into all the major DE tool workflows. You can read more about this problem here. </p>

To reduce these bias, there are a lot of methods to normalize RNAseq data.

<p>Those which I call naive ones :

<ul>
    <li>Total count</li>
    <li>Upper Quartile </li>
    <li>RPKM (Reads Per Kilobase per Million, which is not solid enought for cross condition experiment, pub4 & pub5)</li>
</ul>
</p>

Those with a statistical power :

For the batch effect
<ul>
  <li>RLE method (Relative log Expression) like DESeq2 </li>
  <li>TMM method (Trimmed Mean of M values) like edgeR </li>
</ul>
    Plus, the most used rule to normalize gene count :

<ul> 
    <li>negative binomial distribution (edgeR, DESeq2)</li>
<li>Add to that a multiple testing correction, to output strong express genes (DESeq2)</li>
</ul>
    Library size is the major biais and could be handle in DESeq2 using the <b> sizeFactor </b>

In [ ]:
##--------------------
##   RAW VARIABLES
##--------------------
y.raw          <- y
ngroups        <- length(unique(y$samples$group)); ngroups
nsamples       <- ncol(y); nsamples
ngenes         <- nrow(y); ngenes

raw_lib        <- y$samples; head(raw_lib)
raw_counts     <- y$counts; #dim(raw_counts)
raw_cpm        <- cpm(y, prior.count=2);# dim(raw_cpm)
raw_lcpm       <- cpm(y, prior.count=2, log=TRUE);# dim(raw_lcpm)
raw_anno       <- y$genes; head(raw_anno)
raw_L          <- log2(2/(mean(y$samples$lib.size) * 1e-06)); #raw_L

<b>Within-Sample Normalization = Counts per Million (CPM) </b> Read count routinely refers to the number of reads that align to a particular region. Counts per million mapped reads are counts scaled by the number of sequenced fragments multiplied by one million. <b>Transcripts per million (TPM)</b> is a measurement of the proportion of transcripts in a pool of RNA.

In [ ]:
pearson.raw_lcpm      <- cor(raw_lcpm, use="all.obs", method="pearson")
spearman.raw_lcpm     <- cor(raw_lcpm, use="all.obs", method="spearman")

raw_var     <- matrixStats::rowVars(raw_lcpm) # row variance per gene
raw_scale   <- t(scale(t(raw_lcpm)))  # scaled lcpm values mean=0, std=1

stopifnot(rownames(raw_scale)==rownames(y$genes))
stopifnot(rownames(raw_scale)==rownames(y$counts))

In [ ]:
## mean, median, min., and max. library sizes
L  <- mean(y$samples$lib.size) * 1e-06
M  <- median(y$samples$lib.size) * 1e-06
Mn <- min(y$samples$lib.size) * 1e-06
Mx <- max(y$samples$lib.size) * 1e-06
round(c(L, M, Mn, Mx),1)

# smallest libraries
min_group <- min(table(targets$group))  
min_group
 
# calculates the CPM value that corresponds to a count of 10
cpm(10, min(y$samples$lib.size))
min_cpm <- round(as.numeric(cpm(10, min(y$samples$lib.size))),2)
min_cpm 

keep <- rowSums(cpm(y) > min_cpm) >=  min_group
table(keep)

y <- y[keep, , keep.lib.sizes=FALSE]
dim(y) 


#### The option **keep.lib.sizes=FALSE** causes the library sizes to be recomputed after filtering.
#### This is generally recommended, although the effect on the downstream analysis is usually small.

In [ ]:
##---------------------------
## FILTER VARIABLES (y)
##---------------------------
y.filter           <- y
filter_lib         <- y$samples
filter_ngenes      <- nrow(y); filter_ngenes
filter_anno        <- y$genes; head(filter_anno)
head(filter_anno)[1:5,]; dim(filter_anno)
filter_counts      <- y$counts
filter_cpm         <- cpm(y, prior.count=2)
filter_lcpm        <- cpm(y, prior.count=2, log=TRUE)
filter_L           <- log2(2/(mean(y$samples$lib.size) * 1e-6)); filter_L 

pearson.filter_lcpm      <- cor(filter_lcpm, use="all.obs", method="pearson")
spearman.filter_lcpm     <- cor(filter_lcpm, use="all.obs", method="spearman")

filter_var     <- matrixStats::rowVars(filter_lcpm) # row variance per gene
filter_scale   <- t(scale(t(filter_lcpm)))  # scaled lcpm values mean=0, std=1

print("Saved filtered variables")

stopifnot(rownames(filter_scale)==rownames(y$genes))
stopifnot(rownames(filter_scale)==rownames(y$counts))

<b>Between sample normalization - Trimmed mean of M values (TMM) </b>
<p><b> CPM "normalization" </b> accounts for library size differences between samples, and produces normalized values that can be compared on an absolute scale (e.g., for filtering). <b>TMM normalization </b> accounts for composition bias, and computes normalization factors for comparing between libraries on a relative scale. <b>CPM</b> normalization doesn't account for composition bias, and <b>TMM</b> normalization doesn't produce normalized values. Thus, you need both steps in the analysis pipeline.  </p>

<b>Reference :</b> </br>
Robinson, M.D., Oshlack, A. A scaling normalization method for differential expression analysis of RNA-seq data. Genome Biol 11, R25 (2010). https://doi.org/10.1186/gb-2010-11-3-r25 </br>
https://support.bioconductor.org/p/69433/

In [ ]:
##-------------------------------
##     TMM NORMALIZATION (y)     
##-------------------------------

# NORMALIZED DATA (y)
y <- calcNormFactors(y, method="TMM")
head(y$samples)

##---------------------------
##   NORM VARIABLES (y)
##---------------------------
y.norm          <- y
norm_lib        <- y$samples
norm_lib        <- cbind(y$samples, (y$samples$norm.factors * y$samples$lib.size))
colnames(norm_lib) <- c("group", "old.lib.size", "norm.factors", "lib.size"); head(norm_lib)

norm_factor     <- y$samples$norm.factors
norm_anno       <- y$genes
norm_counts     <- y$counts
norm_cpm        <- cpm(y, prior.count=2, normalized.lib.sizes = TRUE)
norm_lcpm       <- cpm(y, prior.count=2, normalized.lib.sizes = TRUE, log=TRUE)
norm_L          <- log2(2/(mean(y$samples$lib.size, normalized.lib.sizes=TRUE) * 1e-06)); norm_L 

pearson.norm_lcpm      <- cor(norm_lcpm, use="all.obs", method="pearson")
spearman.norm_lcpm     <- cor(norm_lcpm, use="all.obs", method="spearman")

norm_var     <- matrixStats::rowVars(norm_lcpm) # row variance per gene
norm_scale   <- t(scale(t(norm_lcpm)))  # scaled lcpm values mean=0, std=1

stopifnot(rownames(norm_scale)==rownames(y$genes))
stopifnot(rownames(norm_scale)==rownames(y$counts))

### Limma voom analysis

**Limma** is an R package that was originally developed for differential expression (DE) of microarray data. </br>
**Voom** is a function in the limma package that modifies RNA-seq data for use with limma. 

1. voom method estimates the mean-variance relationship of the log-counts
2. generates a precision weight for each observation
3. enters these into the limma empirical Bayes analysis

count data shows a marked mean-variance relationship. Raw counts show increasing variance with increasing count size. Log counts typically show a decreasing mean-variance trend. Voom estimates mean-variance trend for log2 cpm counts, then assigns a weight to each observation based on its predicted variance. The weights are then used in the linear modelling process to adjust for heteroscedasticity. 

**Limma-voom** is the tool of choice for DE because it:

1. allows for flexible model specifications such that multiple categorical and continuous variables can be included.
2. based on simulation studies, maintains the false discovery rate at or below the nominal rate
3. Empirical Bayes smoothing of gene-wise standard deviations provides increased power.

**References:** 
Law, C.W., Chen, Y., Shi, W. et al. voom: precision weights unlock linear model analysis tools for RNA-seq read counts. Genome Biol 15, R29 (2014). https://doi.org/10.1186/gb-2014-15-2-r29 </br>
https://rdrr.io/bioc/limma/man/voom.html </br>
https://ucdavis-bioinformatics-training.github.io/2018-June-RNA-Seq-Workshop/thursday/DE.html

In [ ]:
y$samples
y$samples$group

### Specify the model to be fitted. We do this before using voom since voom uses variances of the model residuals (observed - fitted)
#### a model where each coefficient corresponds to a group mean

In [ ]:
#design = model.matrix(~0 + my_targets$group + my_targets$batch,  data= y$samples)
design = model.matrix(~0 + y$samples$group,  data= y$samples)
design

In [ ]:
#colnames(design) <- c("Hrt_ctrl","Hrt_drug");design
colnames(design) <- c("Msle_ctrl","Msle_drug");design

In [ ]:
#contrasts <- makeContrasts(Drug_vs_Ctrl=Hrt_drug-Hrt_ctrl, levels=colnames(design));contrasts
contrasts <- makeContrasts(Drug_vs_Ctrl=Msle_drug-Msle_ctrl, levels=colnames(design));contrasts

In [ ]:
v <- voom(y, design, plot=FALSE)
vfit<-lmFit(v, design)
vfit<-contrasts.fit(vfit,contrasts=contrasts)
results <-eBayes(vfit)

## What is voom doing?

<ul>
    <li>Counts are transformed to log2 counts per million reads (CPM), where “per million reads” is defined based on the normalization factors we calculated earlier</li>
    <li>A linear model is fitted to the log2 CPM for each gene, and the residuals are calculated </li>
    <li>A smoothed curve is fitted to the sqrt(residual standard deviation) by average expression (see red line in plot above) </li>
    <li>The smoothed curve is used to obtain weights for each gene and sample that are passed into limma along with the log2 CPMs. </li>
</ul>

#### add figure

The above is a “good” voom plot. If your voom plot looks like the below, you might want to filter more:

### Empirical Bayes smoothing of standard errors
#### shrinks standard errors that are much larger or smaller than those from other genes towards the average standard error

## Reference:
https://genomebiology.biomedcentral.com/articles/10.1186/gb-2014-15-2-r29 <br />
https://www.degruyter.com/doi/10.2202/1544-6115.1027


In [ ]:
png(file.path(".", "01_voom.png"),units="in", width=10, height=5, res=1000, pointsize=12)
par(mfrow=c(1,2))
   voom(y,design=design, normalize.method="none", plot=TRUE, save.plot=TRUE)
   plotSA(results, main="Final Model")
dev.off()

In [ ]:
plotSA(results, main="Final Model")

In [ ]:
stats <- topTable(results, coef=1, n=Inf, adjust.method="BH", sort.by="none", p.value=1, lfc=0); dim(stats)
results$stats <- stats
sig <- topTable(results, coef=1, n=Inf, adjust.method="BH", sort.by="none", p.value=0.05, lfc=1); dim(sig)
dt <- decideTests(results, method="separate", adjust.method="BH", p.value=0.05, lfc = 1)
summary(dt)


In [ ]:
# save results

write.csv(stats,"stats.csv")
write.csv(sig,"sig.csv")
write.csv(summary(dt), "dt_summary.csv")

<ul>
    <li> <b>logFC:</b> log2 fold change of drug/control </li>
    <li> <b>AveExpr:</b> Average expression across all samples, in log2 CPM </li>
    <li> <b>t:</b> logFC divided by its standard error </li>
    <li> <b>P.Value:</b> Raw p-value (based on t) from test that logFC differs from 0 </li>
    <li> <b>adj.P.Val:</b> Benjamini-Hochberg false discovery rate adjusted p-value </li>
    <li> <b>B:</b> log-odds that gene is DE (arguably less useful than the other columns) </li>
</ul>

In [ ]:
png(file.path(".", paste("05_","drug_vs_CON", "_p-value.histogram.png",sep="")), units="in", width=6, height=5, res=1000, pointsize=12)
{
   grayblue <-"#b0ddf5"
      
     hist(results$stats$P.Value, col= grayblue, breaks = 100, 
          main=paste("P-value Histogram: ","drug vs. CON",sep=""),
          xlab="P.Value", 
          ylab="frequency", 
          font=1, cex.main=1, las=1)
}
dev.off()

In [ ]:
     hist(results$stats$P.Value, col= grayblue, breaks = 100, 
          main=paste("P-value Histogram: ","drug vs. CON",sep=""),
          xlab="P.Value", 
          ylab="frequency", 
          font=1, cex.main=1, las=1)


In [ ]:
i<-1

png(file.path(".", paste("07_","Drug_vs_CON","_MD.plot.png",sep="")), units="in", width=5, height=5, res=1000, pointsize=10)
{
     ##----------------------------------------------------------------------------
     op <- par(no.readonly = TRUE)  # save current par settings b4 plotting
     #par(mfrow = c(4, 5))          # multiple plots
     par(mar = c(5, 5, 4, 1))       # inner margin (bottom, left, top, right)
     par(mgp = c(3, 1, 0))          # axis margin (labels, ticks, line)
     par(oma = c(2, 1, 0, 1))       # outer margin (bottom, left, top, right)
     ##----------------------------------------------------------------------------
     plotMD(results, column=i, cex=1, las=1, status=dt[,i], values=c(1,-1), col=c("red","blue"), 
            legend="topright", main="", 
            xlab=expression(paste("avg. ", log[2]," (counts-per-million)",sep="")),
            ylab=expression(paste(log[2]," (fold-change)",sep="")),
            cex.axis=1.1, cex.lab=1.2
            # ,ylim=c(-11,13)
            #,xlim=c(2,14)
     )
     title(main=paste("MD plot: ", "Drug vs. CON", "", sep=""))
     legend("topright", c("up", "not sig.", "down"), inset=0, pch=19,
            box.col="black", box.lwd=1, bg="white",
            col=c("red","black","blue"), ncol=1, cex=1)
     
     # text(x=12, y=-3, label=paste("up =", summary(dt)[3], sep=" "), adj=0, col="red", cex=0.9, font=1)
     # text(x=12, y=-4, label=paste("not sig. =", summary(dt)[2], sep=" "), adj=0, col="black", cex=0.9, font=1)
     # text(x=12, y=-5, label=paste("down =", summary(dt)[1], sep=" "), adj=0, col="blue", cex=0.9, font=1)
     abline(h=c(-1,1), col="black", lty=2,lwd=1)
}


dev.off()


In [ ]:
i<-1

     ##----------------------------------------------------------------------------
     op <- par(no.readonly = TRUE)  # save current par settings b4 plotting
     #par(mfrow = c(4, 5))          # multiple plots
     par(mar = c(5, 5, 4, 1))       # inner margin (bottom, left, top, right)
     par(mgp = c(3, 1, 0))          # axis margin (labels, ticks, line)
     par(oma = c(2, 1, 0, 1))       # outer margin (bottom, left, top, right)
     ##----------------------------------------------------------------------------
     plotMD(results, column=i, cex=1, las=1, status=dt[,i], values=c(1,-1), col=c("red","blue"), 
            legend="topright", main="", 
            xlab=expression(paste("avg. ", log[2]," (counts-per-million)",sep="")),
            ylab=expression(paste(log[2]," (fold-change)",sep="")),
            cex.axis=1.1, cex.lab=1.2
            # ,ylim=c(-11,13)
            #,xlim=c(2,14)
     )
     title(main=paste("MD plot: ", "drug vs. CON", "", sep=""))
     legend("topright", c("up", "not sig.", "down"), inset=0, pch=19,
            box.col="black", box.lwd=1, bg="white",
            col=c("red","black","blue"), ncol=1, cex=1)
     
     # text(x=12, y=-3, label=paste("up =", summary(dt)[3], sep=" "), adj=0, col="red", cex=0.9, font=1)
     # text(x=12, y=-4, label=paste("not sig. =", summary(dt)[2], sep=" "), adj=0, col="black", cex=0.9, font=1)
     # text(x=12, y=-5, label=paste("down =", summary(dt)[1], sep=" "), adj=0, col="blue", cex=0.9, font=1)
     abline(h=c(-1,1), col="black", lty=2,lwd=1)



In [ ]:
library(Glimma, quietly=TRUE)
  
glMDPlot(results, 
         coef=i,
         counts = norm_lcpm, 
         anno = results$genes,
        # groups = my_targets$group,
         groups = y$samples$group,
         samples = colnames(norm_lcpm), 
         status = dt[,i],
         transform=FALSE, 
         main = paste("MD Plot: ","drug vs. CON", "", sep=" "),
         xlab = "avg. log2 (counts-per-million)",
         ylab = "log2 (fold-change)", 
         side.xlab = "group",
         side.ylab = "expression (norm_lcpm)", 
         side.log = FALSE,
         # side.gridstep = ifelse(!transform || side.log, FALSE, 0.5),
         p.adj.method = "BH",
         jitter = 30, 
         side.main = "SYMBOL",
         display.columns = colnames(results$genes), 
         cols = c("#00bfff", "#858585", "#ff3030"),
         #sample.cols = colors[my_targets$group],
         path = file.path("."),
         folder = "MD-plots", 
         html = paste("drug_vs_CON", "_MD.plot", sep=""), 
         launch = FALSE
     )

In [ ]:
png(file.path(".", paste("08_","drug_vs_CON","_volcano.plot.png",sep="")), units="in", width=6, height=6, res=1000, pointsize=12)
{
     with(results$stats, 
          plot(logFC, -log10(adj.P.Val),# pch=20, las=1,
               pch = 21, bg = "black", col = "black", lwd = 0.9, cex = 1,
               main=paste("Volcano Plot: ", "drug vs. CON","", sep=" "), 
               xlab=expression(paste(,log[2]," (fold-change)",sep="")),
               #xlab= "logFC", 
               ylab=expression(paste("-",log[10]," (adj. p-value)",sep=""))
               # ylab="-log10 (P.Value)", 
               # ,ylim= c(0, 8)
               , xlim= c(-2,7)
               , cex.axis=1.1, cex.lab=1.2
          )
     )
     # grid()
     
     with(subset(results$stats, adj.P.Val <= 0.05 & logFC >= 1), points(logFC, -log10(adj.P.Val), pch = 21, bg = "firebrick2", col = "firebrick2", lwd = 1, cex = 1.1))
     with(subset(results$stats, adj.P.Val <= 0.05 & logFC <= -1), points(logFC, -log10(adj.P.Val), pch = 21, bg = "blue", col = "blue", lwd = 1, cex = 1.1))

     abline(v= c(-1,1), col="black", lty=2, lwd=1)
     abline(h= -log10(0.05), col="black", lty=2, lwd=1)
     
     # text(x=-14, y=8, label=paste("up = ", summary(dt)[3], sep=" "), adj=0, col="red", cex=0.8, font=1)
     # text(x=-14, y=7.5, label=paste("not sig = ", summary(dt)[2], sep=" "), adj=0, col="black", cex=0.8, font=1)
     # text(x=-14, y=7, label=paste("down = ", summary(dt)[1], sep=" "), adj=0, col="blue", cex=0.8, font=1)
}

dev.off()

In [ ]:
glXYPlot(x=results$stats$logFC,
         y=-log(results$stats$adj.P.Val, 10),
         counts = norm_lcpm,
         #groups = my_targets$group,
         #samples = rownames(my_targets),
         groups = y$samples$group,
         samples = rownames(y$samples),
         status = dt[,i], 
         transform=FALSE, 
         anno = results$genes,
         display.columns = colnames(results$genes),
         xlab = "log2 (fold-change)",
         ylab = "-log10 (adj. p-value)",
         side.main = "SYMBOL",
         side.xlab = "group",
         side.ylab = "expression (norm_lcpm)",
         side.log=FALSE,
         #sample.cols = colors[my_targets$group],
         cols = c("#00bfff", "#858585", "#ff3030"),
         p.adj.method="BH",
         jitter = 30,
         path = file.path("."),
         folder = "Volcano-plots",
         html = paste("drug_vs_CON","_volcano.plot", sep=""),
         main=paste("Volcano Plot: ","drug vs. CON", "", sep=" "),
         launch = FALSE
         )

In [ ]:
# check have same samples and order
ncol(y$counts)
length(y$samples)
colnames(y$counts)
colnames(norm_lcpm)
rownames(y$samples)

## NOTES

### Question 1: Can you export normalized counts from edgeR DGElist

 If you want to export normalized expression values, just use cpm or rpkm. 
 The root of the confustion is there is no such thing as a **TMM normalized count** because TMM normalizes the library sizes rather than the counts. A normalized value can no longer be a count. 

**TMM** normalizes the library sizes to produce effective library sizes. </br>
**CPM** values are counts normalized by the effective library sizes. </br>
**RPKM** values are counts normalized by effective library sizes and by gene/feature length. 

Secondly, people assume edgeR must be storing **normalized counts** internally, but it does not. Most edgeR DE pipelines never modify the original counts in any way.

#### Normalization for library size is instead implicit as part of the model-fitting. edgeR does not use cpm or rpkm values internally in its DE pipelines, rather they are only for export or for graphical purposes. 

A third source of confusion is that the original edgeR pipeline (**classic** pipeline) did compute pseudo.counts internally, which are equivalent to the original counts but with equalized effective library sizes. 
The pseudo.counts were used only to estimate dispersions, not to assess DE or to compute fold-changes. We did not intend or recommend that users would export these as normalized values but some have done so.  It is not appropriate to multiple pseudo.counts by norm.factors as one blog suggests. 

#### Examples of posts by edgeR authors:
https://support.bioconductor.org/p/133671/ </br>
https://support.bioconductor.org/p/46779/ </br>
https://www.biostars.org/p/9475236/ 


In [ ]:
# download folders from On demand
# open terminal and change to the directory of the files to save. 

zip -r filename.zip /path/directory